## 15.02.2025

The purpose of this coding is to do the data preprocessing for one night data. 
Reference: Local and Widespread Slow Waves in Stable NREM Sleep: Evidence for Distinct Regulation Mechanisms

### Import `.mat` raw data

In [2]:
import numpy as np
import pandas as pd
from scipy.io import loadmat
import os


In [3]:
os.getcwd()

'/home/gs/code/swift-eeg/notebook'

In [4]:
# Data folder

data_folder = "/home/gs/code/swift-eeg/data/raw"

# Load the .mat file
data = loadmat(data_folder + "/Filtered_EPISL_01_W1_EEG_FiltDwn_05to30Hz.mat")

### Load the data. 

The data is in dictionary format with the following keys:
- `__header__` 
- `__version__`
- `__globals__`
- `EEG_filtered`

EEG -> EEG data channel x samples @ 125 Hz, references to Cz

visnum -> indices of expert labeled sleep stages for each 20s epoch

- 1 = awake
- 0 = REM
- -1 = NREM 1
- -2 = NREM 2
- -3 = NREM 3

visgood -> indices of clean epochs

chanlocs -> channel information


The difference of the UZH data with the data from the paper.

- 125 Hz vs 500 Hz
- 20 s per epoch vs 30 s per epoch


In [4]:
3605508 / 125 / 20

1442.2032

In [6]:
# EEG data from the .mat file, shape = (129, 3605508)
eeg = data["EEG_filtered"]["EEG"][0, 0]

# Channel location
channel_location = data["EEG_filtered"]["chanlocs"][0, 0]

# Indices of clean epochs. Adjusted to 0-based indexing, shape = (1, 1267)
clean_epoch_index = data["EEG_filtered"]["visgood"][0, 0] - 1

# Expert labeled sleep stages, shape = (1, 1442)
sleep_stage = data["EEG_filtered"]["visnum"][0, 0]

clean_epoch_index = clean_epoch_index.astype(int)
sleep_stage = sleep_stage.astype(int)

In [ ]:
channel_location

In [14]:
type(channel_location)

numpy.ndarray

In [17]:
channel_location[0, 128]

np.void((array([[0]], dtype=uint8), array([[0]], dtype=uint8), array(['Cz'], dtype='<U2'), array([[-0.]]), array([[90]], dtype=uint8), array([[0]], dtype=uint8), array([[90]], dtype=uint8), array([[6.123234e-17]]), array([[-0.]]), array([[1]], dtype=uint8)), dtype=[('theta', 'O'), ('radius', 'O'), ('labels', 'O'), ('sph_theta', 'O'), ('sph_phi', 'O'), ('sph_theta_besa', 'O'), ('sph_phi_besa', 'O'), ('X', 'O'), ('Y', 'O'), ('Z', 'O')])

Each EEG recording was band-pass fileted (0.5 - 58 Hz), and sleep stage N2 and N3 were extracted

In [7]:
import numpy as np
import mne

# 📌 Load EEG Data
fs = 125 # Sampling frequency in Hz (change if different)
eeg_data = eeg  # Shape: (129 channels, 3,605,508 samples)


# 📌 Step 1: Band-pass Filter (0.5–58 Hz)
def bandpass_filter(eeg_data, lowcut=0.5, highcut=58, fs=fs, order=4):
    """Apply band-pass filter to EEG data"""
    from scipy.signal import butter, filtfilt

    nyquist = 0.5 * fs
    low, high = lowcut / nyquist, highcut / nyquist
    b, a = butter(order, [low, high], btype="band")
    return filtfilt(b, a, eeg_data, axis=1)  # Apply filter across time axis


filtered_eeg = bandpass_filter(eeg_data) # shape = (129, 3605508)

# 📌 Step 2: Extract 30s Epochs of N2 & N3 Sleep Stages
# Assume sleep stage annotations exist in "sleep_stages.npy" (Shape: (n_samples,))
# sleep_stages = np.load("data/raw/sleep_stages.npy")  # Example sleep stage array

# Find indices corresponding to N2 & N3 sleep
n2_n3_indices = np.where((sleep_stage == -2) | (sleep_stage == -3))[1] # shape = (935,)

selected_indices = np.intersect1d(n2_n3_indices, clean_epoch_index.squeeze())


In [8]:
selected_indices

array([  29,   30,   31,   32,   33,   34,   35,   36,   37,   38,   39,
         40,   41,   42,   43,   44,   45,   46,   47,   48,   49,   50,
         51,   52,   53,   54,   55,   56,   57,   58,   59,   60,   61,
         62,   63,   64,   65,   66,   67,   68,   69,   70,   71,   72,
         73,   74,   76,   77,   78,   79,   80,   81,   82,   83,   84,
         85,   86,   87,   88,   89,   90,   91,   92,   93,   94,   95,
         96,   97,   98,   99,  100,  101,  102,  103,  104,  105,  106,
        107,  108,  109,  110,  111,  112,  113,  114,  115,  116,  117,
        118,  119,  120,  121,  122,  123,  124,  125,  126,  127,  128,
        129,  130,  133,  134,  135,  136,  137,  138,  139,  140,  141,
        142,  143,  144,  145,  146,  147,  148,  149,  150,  151,  152,
        153,  154,  155,  156,  157,  158,  159,  160,  161,  164,  165,
        166,  167,  168,  169,  170,  171,  172,  173,  174,  175,  178,
        180,  181,  183,  184,  185,  186,  187,  1

In [9]:

# Extract 30s epochs (20s * fs samples)
epoch_length = fs * 20  # 20s epochs
selected_epochs = [
    filtered_eeg[:, i : i + epoch_length]
    for i in selected_indices
    if i + epoch_length < filtered_eeg.shape[1]
]

In [10]:

# Convert to NumPy array (Shape: (n_epochs, n_channels, epoch_length))
selected_epochs = np.array(selected_epochs)

# 📌 Step 3: Remove Bad Channels & Interpolate Using MNE
raw = mne.io.RawArray(
    filtered_eeg,
    mne.create_info(
        ch_names=[f"Ch{i}" for i in range(eeg_data.shape[0])], sfreq=fs, ch_types="eeg"
    ),
)

# Manually define bad channels (Example: Channels 10 & 20 are bad)
# bad_channels = ["Ch10", "Ch20"]
# raw.info["bads"] = bad_channels

# Interpolate bad channels using spherical splines
# raw.interpolate_bads(reset_bads=True, mode="accurate")

# 📌 Step 4: Apply ICA to Remove Artifacts
ica = mne.preprocessing.ICA(n_components=30, random_state=97, method="fastica")
ica.fit(raw)



Creating RawArray with float64 data, n_channels=129, n_times=3605508
    Range : 0 ... 3605507 =      0.000 ... 28844.056 secs
Ready.
Fitting ICA to data using 129 channels (please be patient, this may take a while)


/tmp/ipykernel_372343/3695853375.py:21: RuntimeWarning: The data has not been high-pass filtered. For good ICA performance, it should be high-pass filtered (e.g., with a 1.0 Hz lower bound) before fitting ICA.
  ica.fit(raw)


: 

In [ ]:

# Automatically detect & remove ECG/EOG artifacts
ecg_indices, _ = ica.find_bads_ecg(raw, method="correlation")
eog_indices, _ = ica.find_bads_eog(raw)
ica.exclude = ecg_indices + eog_indices  # Remove bad components
raw = ica.apply(raw)

# 📌 Step 5: Remove 72 Neck/Face Electrodes & Keep 185 Internal Channels
# Assume the indices of face/neck electrodes are in "face_neck_channels.npy"
face_neck_channels = np.load("data/raw/face_neck_channels.npy")  # Shape: (72,)

# Keep only the remaining 185 channels
kept_channels = np.delete(raw.get_data(), face_neck_channels, axis=0)

# 📌 Save Preprocessed EEG Data
np.save("data/preprocessed/eeg_filtered.npy", kept_channels)

# 📊 Plot Raw & Processed EEG Signals
raw.plot(duration=5, n_channels=20, scalings="auto", title="Preprocessed EEG")

In [11]:
import mne

import numpy as np

# Example EEG data: (n_channels, n_samples)
fs = 125  # Sampling frequency (Hz)


# Create MNE RawArray object
info = mne.create_info(
    ch_names=[f"Ch{i}" for i in range(eeg_data.shape[0])], sfreq=fs, ch_types="eeg"
)
raw = mne.io.RawArray(eeg, info)

# Apply band-pass filter (0.5 - 58 Hz)
raw.filter(l_freq=0.5, h_freq=58, fir_design="firwin")

# Convert back to NumPy array (if needed)
filtered_eeg = raw.get_data()

NameError: name 'eeg_data' is not defined